In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [2]:
#List of all counties
counties = ['Yuba County.csv'
, 'Alameda County.csv'
, 'Alpine County.csv'
, 'Amador County.csv'
, 'Butte County.csv'
, 'Calaveras County.csv'
, 'Colusa County.csv'
, 'Contra Costa County.csv'
, 'Del Norte County.csv'
, 'El Dorado County.csv'
, 'Fresno County.csv'
, 'Glenn County.csv'
, 'Humboldt County.csv'
, 'Imperial County.csv'
, 'Inyo County.csv'
, 'Kern County.csv'
, 'Kings County.csv'
, 'Lake County.csv'
, 'Lassen County.csv'
, 'Los Angeles County.csv'
, 'Madera County.csv'
, 'Marin County.csv'
, 'Mariposa County.csv'
, 'Mendocino County.csv'
, 'Merced County.csv'
, 'Modoc County.csv'
, 'Mono County.csv'
, 'Monterey County.csv'
, 'Napa County.csv'
, 'Nevada County.csv'
, 'Orange County.csv'
, 'Placer County.csv'
, 'Plumas County.csv'
, 'Riverside County.csv'
, 'Sacramento County.csv'
, 'San Benito County.csv'
, 'San Bernardino County.csv'
, 'San Diego County.csv'
, 'San Francisco County.csv'
, 'San Joaquin County.csv'
, 'San Luis Obispo County.csv'
, 'San Mateo County.csv'
, 'Santa Barbara County.csv'
, 'Santa Clara County.csv'
, 'Santa Cruz County.csv'
, 'Shasta County.csv'
, 'Sierra County.csv'
, 'Siskiyou County.csv'
, 'Solano County.csv'
, 'Sonoma County.csv'
, 'Stanislaus County.csv'
, 'Sutter County.csv'
, 'Tehama County.csv'
, 'Trinity County.csv'
, 'Tulare County.csv'
, 'Tuolumne County.csv'
, 'Ventura County.csv'
, 'Yolo County.csv']
            
for county in counties:
    rent = pd.read_csv(f"Counties_Rent/{county}")
    rent = rent.rename(columns = {"Unnamed: 0" : "date"})
    inflation = pd.read_csv("inflation.csv")
    rent_df = pd.merge(rent, inflation, on=["date"], how="left")
    rent_df = rent_df.drop(columns=["Unnamed: 0", " Annual Change", ' '])

    # obtian cummalitive inflation

    cummalitive = []
    adjust = []

    #iterate through inflation column
    for rate in rent_df[" Inflation Rate (%)"]:
        # append rate to list
        print(rate)
        cummalitive.append(rate)
    total = 0

    #obtain cummalitive rate for each year 
    while total <= 15:
        rate = 0
        for i in cummalitive[total:]:
            print(rate)
            rate = rate + i
        total = total + 1    
        adjust.append(rate)

    # make cummalitive rate column 
    rent_df["Cummalitive"] = adjust
    rent_df.head()

    #change decimal place
    rent_df["Cummalitive"] =  rent_df["Cummalitive"] * 0.01

    #get rid of commas and $
    rent_df["0 BR"] = rent_df["0 BR"].str.replace("$", "").str.replace(",", "")
    rent_df["1 BR"] = rent_df["1 BR"].str.replace("$", "").str.replace(",", "")
    rent_df["2 BR"] = rent_df["2 BR"].str.replace("$", "").str.replace(",", "")
    rent_df["3 BR"] = rent_df["3 BR"].str.replace("$", "").str.replace(",", "")
    rent_df["4 BR"] = rent_df["4 BR"].str.replace("$", "").str.replace(",", "")
    rent_df["Est. Population"] = rent_df["Est. Population"].str.replace("$", "").str.replace(",", "")

    # changing datatype to int
    rent_df["0 BR"] = rent_df["0 BR"].astype(int)
    rent_df["1 BR"] = rent_df["1 BR"].astype(int)
    rent_df["2 BR"] = rent_df["2 BR"].astype(int)
    rent_df["3 BR"] = rent_df["3 BR"].astype(int)
    rent_df["4 BR"] = rent_df["4 BR"].astype(int)
    rent_df["Est. Population"] = rent_df["Est. Population"].astype(int)

    # adjust prices for inflation
    rent_df["0_BR_adjust"] = rent_df["Cummalitive"] * rent_df["0 BR"] + rent_df["0 BR"]
    rent_df["1_BR_adjust"] = rent_df["Cummalitive"] * rent_df["1 BR"] + rent_df["1 BR"]
    rent_df["2_BR_adjust"] = rent_df["Cummalitive"] * rent_df["2 BR"] + rent_df["2 BR"]
    rent_df["3_BR_adjust"] = rent_df["Cummalitive"] * rent_df["3 BR"] + rent_df["3 BR"]
    rent_df["4_BR_adjust"] = rent_df["Cummalitive"] * rent_df["4 BR"] + rent_df["4 BR"]

    # list of bredroom columns
    columns = ["0 BR", "1 BR", "2 BR", "3 BR", "4 BR"]
    names = rent_df.columns.tolist()[6:]
    row = {}

    #predicted date for lstm
    row["date"] = 2022
    row["Est. Population"] = rent_df["Est. Population"].tolist()[-1]


    for column in columns:    
        # univariate data prep LSTM
        from numpy import array

        # split sequence
        def split_sequence(sequence, n_steps):
            X, y = list(), list()
            for i in range(len(sequence)):
                # find end of pattern 
                end_ix = i + n_steps
                #check if beyond sequence to break
                if end_ix > len(sequence)-1:
                    break
                #gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return array(X), array(y)
        # sequence 
        rent_rate = list(rent_df[column])
        # number of steps
        n_steps = 3
        X, y = split_sequence(rent_rate, n_steps)
        # summarize the data
        for i in range(len(X)):
            print(X[i], y[i])

        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        # fit model
        model.fit(X, y, epochs=200, verbose=0)

        # demonstrate prediction
        x_input = array(rent_df[column][-3:])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        row[column] = yhat.tolist()[0][0]

    # convert to df
    for name in names:
        row[name] = np.nan

    tester = pd.DataFrame.transpose(pd.DataFrame.from_dict(row, orient="index"))

    tester["date"] = tester["date"].astype(int)

    #exports data as csv and json files
    for column in columns:
        tester[column] = tester[column].astype(int) 

    rent_df = rent_df.append(tester)

    rent_df.to_csv(f"Counties_rent_finalized/{county}")
    
    rent_df.set_index('date').to_json(f"Counties_rent_json/{county[-4:]}.json")

3.2259
2.8527
3.8391
-0.3555
1.64
3.1568
2.0693
1.4648
1.6222
0.1186
1.2616
2.1301
2.4426
1.8122
1.2336
4.6979
0
3.2259
6.0786
9.9177
9.5622
11.202200000000001
14.359000000000002
16.4283
17.8931
19.5153
19.6339
20.895500000000002
23.0256
25.4682
27.2804
28.514
0
2.8527
6.691800000000001
6.3363000000000005
7.9763
11.1331
13.2024
14.667200000000001
16.2894
16.408
17.669600000000003
19.7997
22.2423
24.0545
25.2881
0
3.8391
3.4836
5.1236
8.2804
10.3497
11.8145
13.4367
13.5553
14.8169
16.947
19.389599999999998
21.2018
22.435399999999998
0
-0.3555
1.2845
4.4413
6.5106
7.9754000000000005
9.5976
9.7162
10.9778
13.1079
15.550500000000001
17.3627
18.5963
0
1.64
4.7968
6.8661
8.3309
9.9531
10.0717
11.3333
13.4634
15.906
17.7182
18.9518
0
3.1568
5.226100000000001
6.690900000000001
8.3131
8.431700000000001
9.6933
11.823400000000001
14.266000000000002
16.078200000000002
17.3118
0
2.0693
3.5341000000000005
5.156300000000001
5.274900000000001
6.5365
8.6666
11.109200000000001
12.921400000000002
14.1550

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_9138/1756378236.py:97: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rent_df["0 BR"] = rent_df["0 BR"].str.replace("$", "").str.replace(",", "")
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_9138/1756378236.py:98: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rent_df["1 BR"] = rent_df["1 BR"].str.replace("$", "").str.replace(",", "")
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_9138/1756378236.py:99: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when rege

[437 452 510] 534
[452 510 534] 550
[510 534 550] 564
[534 550 564] 618
[550 564 618] 526
[564 618 526] 512
[618 526 512] 551
[526 512 551] 628
[512 551 628] 663
[551 628 663] 674
[628 663 674] 670
[663 674 670] 838
[674 670 838] 875
[493 510 575] 602
[510 575 602] 620
[575 602 620] 636
[602 620 636] 697
[620 636 697] 634
[636 697 634] 617
[697 634 617] 664
[634 617 664] 661
[617 664 661] 679
[664 661 679] 678
[661 679 678] 674
[679 678 674] 844
[678 674 844] 881
[606 627 707] 740
[627 707 740] 763
[707 740 763] 782
[740 763 782] 857
[763 782 857] 812
[782 857 812] 790
[857 812 790] 850
[812 790 850] 861
[790 850 861] 886
[850 861 886] 887
[861 886 887] 878
[886 887 878] 1087
[ 887  878 1087] 1122
[ 882  912 1029] 1077
[ 912 1029 1077] 1110
[1029 1077 1110] 1138
[1077 1110 1138] 1247
[1110 1138 1247] 1173
[1138 1247 1173] 1142
[1247 1173 1142] 1228
[1173 1142 1228] 1255
[1142 1228 1255] 1290
[1228 1255 1290] 1290
[1255 1290 1290] 1269
[1290 1290 1269] 1568
[1290 1269 1568] 1606
[ 944  

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_9138/1756378236.py:186: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rent_df = rent_df.append(tester)


OSError: Cannot save file into a non-existent directory: 'Counties_rent_json'